<a href="https://colab.research.google.com/github/byiringiroscar/NLP-Data-science-ML-and-Artificial-Intelligence/blob/main/Scrap_linkedin_profile_with_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.DKRHxCt1d5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.pfU8jZdusS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.QJsOIkiZK9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [10]:
def contains_linkedin(url):
    return "linkedin" in url and url.startswith('https://rw.linkedin.com/in/')


In [12]:
# Go to the Google homepage
driver.get("https://www.google.com/")

# Find the search bar and enter the search term
key_word = 'linkedin'
word = 'ishimwe diane'
search_bar = driver.find_element(By.NAME, 'q')
search_bar.send_keys(f"{word} {key_word}")
path = "//input[@type='submit']"

# Wait for the search button to be clickable
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, path)))

# Click the search button

# search_button = driver.find_element(By.XPATH, path)
search_button.click()

# Wait for the search results to load
driver.implicitly_wait(10)

# Extract the search results
search_results = driver.find_elements(By.CLASS_NAME, "MjjYud")
try:
  # Print the titles of the search results
  for result in search_results:
      url_searched = result.find_element(By.TAG_NAME, "a")
      url_searched = url_searched.get_attribute("href")
      if contains_linkedin(url_searched):
        print(url_searched)
except:
  pass


# Close the browser window
# driver.quit()

https://rw.linkedin.com/in/diane-ishimwe-644a3a221
https://rw.linkedin.com/in/ishimwe-diane-404a9b17a
https://rw.linkedin.com/in/ishimwe-diane-6a29aa285
https://rw.linkedin.com/in/ishimwe-diane-562a67265
https://rw.linkedin.com/in/ishimwe-diane-07a819228
https://rw.linkedin.com/in/ishimwe-diane-4aa9a110a
https://rw.linkedin.com/in/ishimwe-diane-3a4510276
https://rw.linkedin.com/in/ishimwe-diane-3b0737225?trk=people-guest_people_search-card


In [5]:
def contains_linkedin(url):
    return "linkedin" in url

# Example usage:
url = "https://www.kkkkkk.com/in/ksayinzoga/"
if contains_linkedin(url):
    print("The URL contains 'linkedin'.")
else:
    print("The URL does not contain 'linkedin'.")


The URL does not contain 'linkedin'.
